# 03_answer_with_gemini.ipynb — Tạo câu trả lời bằng Gemini từ top-k context
Notebook này minh hoạ cách:
1) Lấy **top-k context** từ Chroma (dense hoặc hybrid),
2) Gọi **Gemini** (thông qua `src/llm/gemini_client.py`) để sinh câu trả lời **chỉ dựa trên ngữ cảnh**,
3) Trả lời kèm **citation (tr. X, Chương: Y)**.

> ⚠️ Yêu cầu:
> - Bạn đã cài `google-generativeai` và có `GOOGLE_API_KEY` trong `.env`.
> - Đã ingest dữ liệu vào collection (VD: `prisoners_of_geography`).


## 0. Cấu hình
- `COLLECTION_NAME`: tên collection.
- `TOP_K`: số context đưa vào Gemini.
- `MODEL_NAME`: model Gemini muốn dùng (ví dụ `gemini-1.5-flash` để nhanh/tiết kiệm).


In [ ]:
COLLECTION_NAME = 'prisoners_of_geography'
TOP_K = 5
MODEL_NAME = 'gemini-1.5-flash'  # hoặc 'gemini-1.5-pro' nếu bạn muốn chất lượng cao hơn

## 1. Nạp `.env` & import
- Load biến môi trường (đặc biệt `GOOGLE_API_KEY`).
- Import client Chroma và hàm gọi Gemini.


In [ ]:
from dotenv import load_dotenv
_ = load_dotenv()

import os
from src.utils.chroma_client import get_chroma_collection
from src.llm.gemini_client import answer_with_context_gemini

# Kiểm tra đã có GOOGLE_API_KEY chưa (nếu gọi Gemini)
api = os.getenv('GOOGLE_API_KEY', '')
assert api, "GOOGLE_API_KEY trống. Điền key vào .env trước khi chạy cell gọi Gemini."

col = get_chroma_collection(COLLECTION_NAME)

## 2. Chuẩn bị câu hỏi & lấy top-k context (dense)
- Sử dụng `collection.query(query_texts=[q], n_results=TOP_K)`.
- Chuyển đổi `documents + metadatas` → danh sách context phù hợp cho `answer_with_context_gemini`.


In [ ]:
# Đặt câu hỏi tại đây
question = "Ví dụ: Địa hình và vị trí của Nga ảnh hưởng thế nào đến chiến lược của nước này?"  # thay bằng câu của bạn

result = col.query(query_texts=[question], n_results=TOP_K)
docs = result['documents'][0]
metas = result['metadatas'][0]

contexts = []
for d, m in zip(docs, metas):
    contexts.append({
        'text': d,
        'page': f"{m.get('start_page')}→{m.get('end_page')}",
        'chapter': m.get('chapter', '')
    })

len(contexts), contexts[0].keys()

## 3. Gọi Gemini để sinh câu trả lời có citation
- Hàm `answer_with_context_gemini(question, contexts, model_name)` sẽ:
  - Chuẩn hoá ngữ cảnh, 
  - Gửi prompt hệ thống **"chỉ dùng ngữ cảnh"**,
  - Trả về câu trả lời kèm citation.


In [ ]:
answer = answer_with_context_gemini(question, contexts, model_name=MODEL_NAME)
print(answer if answer else "Không nhận được câu trả lời từ Gemini (kiểm tra key/hạn mức).")

## 4. (Tuỳ chọn) Hybrid trước khi gọi Gemini
- Nếu bạn đã có `hybrid_retrieve`, có thể dùng hybrid để lấy context sau đó mới gọi Gemini.


In [ ]:
try:
    from src.rag_pipeline import hybrid_retrieve
    hits = hybrid_retrieve(question, k_dense=TOP_K, k_sparse=TOP_K, collection_name=COLLECTION_NAME)
    # Quy đổi sang contexts cho Gemini
    hybrid_contexts = []
    for h in hits:
        m = h.get('meta', {})
        hybrid_contexts.append({
            'text': h['text'],
            'page': f"{m.get('start_page')}→{m.get('end_page')}" if m else '',
            'chapter': m.get('chapter', '') if m else ''
        })
    print(f"Hybrid contexts: {len(hybrid_contexts)} đoạn")    
    answer2 = answer_with_context_gemini(question, hybrid_contexts, model_name=MODEL_NAME)
    print("\n---\nCâu trả lời (Hybrid):\n", answer2 if answer2 else "Không nhận được câu trả lời từ Gemini.")
except Exception as e:
    print('Bỏ qua hybrid do lỗi hoặc chưa triển khai:', e)

---
### Mẹo
- Nếu câu trả lời dài dòng hoặc có dấu hiệu "bịa", hãy giảm `TOP_K`, tăng chất lượng context (đặt câu hỏi cụ thể hơn), và đảm bảo prompt hệ thống yêu cầu **chỉ dựa trên context**.
- Kiểm tra lại `eval/chapter_map.json` để citation hiển thị tên chương chính xác.
